In [1]:
cd ..

c:\Users\Josu\Documents\Workspace\Human-Benchmark\src\server\model\scripts


In [2]:
import asyncio
import glob
import pathlib
import threading

import aiofiles
import numpy as np
from chess import engine, pgn
from scipy import special

from utils import bitboard

In [3]:
async def worker(path: pathlib.PurePath, semaphore, *, checkpoint: int = 10000):

    def save_checkpoint():
        np.savez(
            f"../data/npz/{path.stem}",
            **kwds
        )

    async with semaphore:
        kwds = {"X": [], "y": []}
        _, simple_engine = await engine.popen_uci(
            "../lib/stockfish-11-win/stockfish-11-win/Windows/stockfish_20011801_x64_modern"
        )
        async with aiofiles.open(path) as f:
            i = 0
            while True:
                game = pgn.read_game(f._file)
                if not game:
                    break
                mainline = game.mainline()
                if mainline:
                    board = np.random.choice(list(mainline)).board()
                    kwds["X"].append(
                        bitboard(board)
                    ),
                    kwds["y"].append(
                        special.expit(
                            (await simple_engine.analyse(
                                board,
                                engine.Limit(depth=15)
                            ))["score"].relative.score(mate_score=32768) / 100
                        )
                    )
                    i += 1
                    if i % checkpoint == 0:
                        save_checkpoint()
        await simple_engine.quit()
        save_checkpoint()

In [4]:
async def main():
    semaphore = asyncio.Semaphore(3)
    await asyncio.gather(*(
        asyncio.ensure_future(
            worker(pathlib.PurePath(file), semaphore)
        )
        for file in glob.glob(
            "../data/*.pgn"
        )
    ))

In [5]:
def run(main, *, debug=False):
    asyncio.set_event_loop_policy(
        engine.EventLoopPolicy()
    )
    asyncio.run(
        main,
        debug=debug
    )

In [6]:
threading.Thread(
    target=run,
    args=(main(),)
).start()